In [36]:
import pandas as pd
from gensim.models.word2vec import Word2Vec

In [37]:
train_df = pd.read_pickle("train.pkl")
test_df = pd.read_pickle("test.pkl")
train_df.head()

,text,category
0,"[想問, 一下, 網路, 消息, 公布, 正取, 報到, 消息, 月, 20, 日, 交叉,...",7
1,"[背景, 畢業, 台南, 大學前, 台南, 師院, 生物科技, 系系排, 25, 大學時, ...",7
2,"[spanclassf2, 引述, mindy30513, 語靜, 之銘言, spanspa...",7
3,[ahrefhttpoaashuedutwuploadrec2012041917105239...,7
4,"[原, PO, 帳號, 資格, 不夠, 幫, PO, 心得, 以下, 帶入, 旁觀者, 看法...",7


In [38]:
corpus = pd.concat([train_df.text, test_df.text]).sample(frac=1)
corpus.head()

3163    [公司, 名稱, 必填, 國立, 成功, 大學, 資源系, 向性, ㄧ, 老師, 實驗室, ...
2001    [跨, 時代, 兩個, 不相干, 標語, 排列, 和平, 奮鬥救, 中國, 快樂, 希望, ...
7376    [網誌, 好, 讀版, ahrefhttpcornidboxcomdiaryread8604...
1704    [包括, HenryCavill, 飾演, Superman, 陽光, 下, 樣子, 飛行,...
2118    [路上, 總是, 遇到, 一些, 擦, 撞然, 後, 遇到, 挑釁, 下車, 攔車, 狀況,...
Name: text, dtype: object

In [39]:
model = Word2Vec(corpus)

In [40]:
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [46]:
most_similar(model, ['懷孕', '網拍', '補習', '東京', 'XDD', '金宇彬','化妝品', '奧斯卡', '主管', '女孩'])

,懷孕,cos,網拍,cos,補習,cos,東京,cos,XDD,cos,金宇彬,cos,化妝品,cos,奧斯卡,cos,主管,cos,女孩,cos
0,出生,0.925556,這雙鞋,0.950963,班,0.967635,北海道,0.964091,XDDD,0.933586,李鍾碩,0.988339,保養品,0.959836,入圍,0.986570,HR,0.944582,兒,0.857643
1,交往,0.911611,韓貨,0.948944,班教,0.926377,大阪,0.957858,XDDDD,0.901040,特講,0.978184,底妝產品,0.948797,獎,0.981072,離職,0.905456,朋友圈,0.829226
2,大寶,0.902683,下標,0.947976,暑假,0.922618,關西,0.941350,XD,0.888754,殷志源,0.978065,好物,0.948055,光速,0.980844,談,0.901534,妹妹,0.814655
3,我爸,0.899110,賣場,0.945829,聽課,0.912034,名古屋,0.936135,忍不住,0.878234,RunningMan,0.977323,NARS,0.946871,PI,0.980675,人資,0.901450,女生,0.811367
4,冷語,0.896684,淘寶的,0.944829,課,0.907499,城,0.929589,XDDDDD,0.867139,目將,0.977278,櫃上,0.946649,最佳影片,0.980431,面談,0.901023,男生,0.794508
5,住院,0.891602,生火,0.936241,補經濟,0.904942,京都,0.918961,很想,0.861651,銀赫,0.976181,回購,0.939220,提名,0.980423,offer,0.893786,惹,0.789557
6,到現,0.891145,貨的,0.935958,三下,0.903654,賞櫻,0.918036,懶,0.854192,有望,0.976084,6g,0.936312,德利,0.980364,面試,0.884242,奔放,0.787924
7,生病,0.889247,買過,0.935555,念,0.903152,札幌,0.917431,我還,0.845350,tvN,0.976028,戀法,0.936282,音樂劇,0.979968,傳產,0.874997,瘋,0.778564
8,放假,0.886690,starmimi,0.931057,大三,0.901133,天滿宮,0.913901,每次,0.841799,主持,0.974707,精華露,0.935585,戰記,0.978769,應徵,0.863219,姐姐,0.777917
9,一歲,0.880918,這雙,0.929140,這一科,0.899497,神戶,0.911810,欸,0.840334,2NE1,0.974242,好用,0.934728,刺客,0.978245,待遇,0.860267,西裝褲,0.775530


In [42]:
model.save('word2vec.model')
model = Word2Vec.load('word2vec.model')

### size=100：詞向量的維度大小，維度太小會無法有效表達詞與詞的關係，維度太大會使關係太稀疏而難以找出規則 

Kaggle比賽上常用的詞向量維度介於200到300之間，在此我們先折衷使用size=250

### iter=5：訓練的回數，訓練過少會使得詞關係過為鬆散，訓練過度又會使得詞關係過為極端

當使用較大的詞向量維度時，可能會需要訓練更多次，我們先用iter=10來看看結果
（實際訓練時，iter只差1就有不小影響，請務必微調這個參數）


In [43]:
model_d250 = Word2Vec(corpus, size=250, iter=10)
most_similar(model_d250, ['懷孕', '網拍', '補習', '東京', 'XDD', '金宇彬','化妝品', '奧斯卡', '主管', '女孩'])

,懷孕,cos,網拍,cos,補習,cos,東京,cos,XDD,cos,金宇彬,cos,化妝品,cos,奧斯卡,cos,主管,cos,女孩,cos
0,懷,0.862840,賣場,0.854013,班,0.880665,北海道,0.877904,XDDD,0.869224,李鍾碩,0.955277,保養品,0.856646,影帝,0.958548,HR,0.891434,兒們,0.665987
1,第一胎,0.846466,韓貨,0.851239,班教,0.820448,造訪,0.842060,XDDDD,0.771904,銀赫,0.937751,專櫃,0.849803,主演,0.945568,面談,0.872589,美女,0.658035
2,孕前,0.842270,彤彤,0.846515,題庫,0.801464,賞櫻,0.836638,忍不住,0.736742,允,0.935111,開架,0.849345,新作,0.940606,人資,0.817667,惹,0.647664
3,第二胎,0.821828,生火,0.845275,正課,0.774268,名古屋,0.836397,XDDDDD,0.735853,RunningMan,0.934044,手邊,0.836755,提名,0.939547,面試,0.810447,帥氣,0.644860
4,孕期,0.815405,淘寶的,0.828878,郭明慶,0.774190,大阪,0.830881,噢,0.705807,殷志源,0.933731,試用,0.822301,執導,0.938182,談,0.794843,男生,0.636913
5,住院,0.810634,買過,0.817406,面授,0.771509,此行,0.827868,XD,0.701087,尹施,0.933466,櫃上,0.820912,德利,0.931436,離職,0.779352,女生,0.635209
6,預產期,0.805002,似款,0.814583,大三,0.770468,橫濱,0.824519,美菜,0.697324,UIE,0.930910,回購,0.805867,獎,0.923570,開會,0.763301,灑花,0.633783
7,寶寶,0.803702,類似款,0.813628,秋季,0.767258,淺草,0.818469,gtlt,0.694261,JTBC,0.928581,好物,0.803078,男星,0.922920,試時,0.762042,打扮,0.632368
8,生病,0.800341,商城,0.813086,自修,0.757124,這趟,0.817915,QQ,0.676487,姜鎬童,0.928484,CHANEL,0.798279,壯士,0.922142,口頭,0.756997,妹妹,0.623533
9,出生,0.797680,Y,0.809123,補帶,0.751871,天滿宮,0.817334,李洪基,0.668307,俞世潤,0.926718,添購,0.795193,奧茲,0.920973,offer,0.754945,男友,0.609175


### sg=0：sg=0時以CBOW來訓練，sg=1時以Skip-gram來訓練
我們這次作業的目標，是要對文章的主題分類作出預測 
某些詞彙可能只出現在特定的主題當中，所以文本中可能有許多低頻詞 
而在特性上，Skip-gram比CBOW通常對低頻詞有更好的訓練效果 
基於以上的猜想，我們可以嘗試用Skip-gram來訓練詞向量，看看能否得到更高的準確度

In [47]:
model_d250_sg = Word2Vec(corpus, size=250, iter=10, sg=1,verbose=True)
most_similar(model_d250_sg, ['懷孕', '網拍', '補習', '東京', 'XDD', '金宇彬','化妝品', '奧斯卡', '主管', '女孩'])

TypeError: __init__() got an unexpected keyword argument 'verbose'

In [ ]:
model_d250_sg = Word2Vec(corpus, size=250, iter=10, sg=0)
most_similar(model_d250_sg, ['懷孕', '網拍', '補習', '東京', 'XDD', '金宇彬','化妝品', '奧斯卡', '主管', '女孩'])

## 以下是其他常用的參數：
### window=5：CBOW下決定Word2Vec一次取多少詞來預測中間詞（Skip-gram的狀況是反過來的）

需要多少詞才能預測中間詞呢？要讓多少詞的含意來影響中間詞的含意呢？
思考完就自己作點實驗吧～（※window只差1就有巨大的影響！請務必微調這個參數）
### min_count=5：出現次數大於等於min_count的詞，才會納入Word2Vec的詞典中

### max_vocab_size=None：Word2Vec的詞典容納上限，出現次數最低的詞會優先被剔除

降低詞典的最大詞數，可能讓模型更容易抓到規則（噪音減少了），但也可能更難抓到規則（認識的詞太少）
### hs=0：hs=0時採用Negative Sampling，hs=1時採用Hierarchical Softmax

### negative=5：Negative Sampling的取樣數量，5~20適合小數據，2~5適合大數據

這會直接影響權重更新的方法，原理有點複雜我不敢亂解釋，請自己去翻論文_(:з」∠)_
### workers=3：訓練用的線程數量（可以加快訓練速度）

In [50]:
from hanziconv import HanziConv
line='你好，这只是个测试'
print(HanziConv.toTraditional(line))

你好，這隻是個測試
